# 毛利率分析报告

## 目标

分析业务的盈利能力，计算整体毛利率和各维度的毛利率分布。

如果在 base44 运行，则 @base44_prompt_mcphub.md。

## 核心指标

**毛利率 = (收入 - 成本) / 收入 × 100%**

- **毛利润** = 收入 - 成本
- **毛利率** = 毛利润 / 收入 × 100%
- 目标: 毛利率持续为正且趋势向上 📈

## 参数

| 参数 | 说明 | 默认值 |
|------|------|--------|
| `start_date` | 开始日期 (YYYY-MM-DD) | 2025-12-13 |
| `end_date` | 结束日期 (YYYY-MM-DD) | 今天 |
| `output_dir` | 输出目录 | ./reports |

## 数据源

**收入表**：
- `user_subscription_stripe_orders` - Stripe 订单
- `user_subscription_paypal_orders` - PayPal 订单
- `apple_used_transactions` - Apple IAP 订单（无 `status` 字段，默认所有交易成功）

**成本表**：
- `art_task` - 任务表，包含 `actual_energy_cost` 字段（单位：美分）

## Agent 运行模式

```bash
jq -r '.cells[] | select(.cell_type=="code") | .source | if type=="array" then join("") else . end' \
  gross-margin-analysis.ipynb > temp/gross-margin-analysis.ts

deno run --no-check --allow-env --allow-net --allow-read --allow-write --allow-ffi \
  temp/gross-margin-analysis.ts 2026-01-13                        # 单天，输出到 ./reports

deno run --no-check --allow-env --allow-net --allow-read --allow-write --allow-ffi \
  temp/gross-margin-analysis.ts 2026-01-11 2026-01-13             # 日期范围

deno run --no-check --allow-env --allow-net --allow-read --allow-write --allow-ffi \
  temp/gross-margin-analysis.ts 2026-01-11 2026-01-13 /tmp/out    # 指定输出目录
```

## Step 0: 环境配置

In [ ]:
const MCP_HUB_URL = Deno.env.get("MCP_HUB_URL") || "http://52.12.230.109:3000/mcp";
const MCP_AUTH_TOKEN = (() => {
  let token = Deno.env.get("MCP_AUTH_TOKEN") || "";
  if (token && !token.startsWith("Bearer ")) token = `Bearer ${token}`;
  return token;
})();
// CLI: deno run ... script.ts <start_date> [end_date] [output_dir]
const OUTPUT_DIR = (typeof Deno !== "undefined" && Deno.args[2]) || "./reports";

let startDate = "";
let endDate = "";
let mcpClient: any = null;

if (!MCP_AUTH_TOKEN) {
  console.error("❌ 请设置环境变量: export MCP_AUTH_TOKEN='your-token'");
} else {
  console.log("✅ 配置加载完成");
}

## Step 1: 连接 MCP Hub

In [13]:
import { Client } from "npm:@modelcontextprotocol/sdk@1.24.3/client/index.js";
import { StreamableHTTPClientTransport } from "npm:@modelcontextprotocol/sdk@1.24.3/client/streamableHttp.js";

const transport = new StreamableHTTPClientTransport(new URL(MCP_HUB_URL), {
  requestInit: { headers: { Authorization: MCP_AUTH_TOKEN } },
});

mcpClient = new Client(
  { name: "gross-margin-analysis", version: "1.0.0" },
  { capabilities: { tools: {}, sampling: {} } }
);

await mcpClient.connect(transport);
console.log("✅ Connected to MCP Hub");

✅ Connected to MCP Hub


## Step 2: 配置分析日期

支持两种模式：
- **CLI 模式**: 从命令行参数获取日期
- **Jupyter 模式**: 通过 prompt 交互输入

In [14]:
const args = typeof Deno !== "undefined" ? Deno.args : [];
const defaultStart = "2025-12-13";
const today = new Date().toISOString().split("T")[0];

if (args.length >= 1) {
  startDate = args[0];
  endDate = args[1] || today;
} else {
  const dateInput = prompt(
    `请输入分析日期（YYYY-MM-DD）\n单天: 2026-01-13\n范围: 2025-12-13,2026-01-13\n默认: ${defaultStart},${today}`,
    `${defaultStart},${today}`
  );
  const parts = (dateInput || `${defaultStart},${today}`).split(",").map(s => s.trim());
  startDate = parts[0];
  endDate = parts[1] || parts[0];
}

const dateList: string[] = [];
let current = new Date(startDate);
const end = new Date(endDate);
while (current <= end) {
  dateList.push(current.toISOString().split("T")[0]);
  current.setDate(current.getDate() + 1);
}

console.log(`✅ 分析日期: ${startDate} 至 ${endDate} (共 ${dateList.length} 天)`);

✅ 分析日期: 2026-01-10 至 2026-01-14 (共 5 天)


## Step 3: 查询收入

从三个收入来源分别查询：

1. **Stripe**: 直接使用 `amount` 字段
2. **PayPal**: ENERGY 类型订单需根据 `paypal_price_id` 确定金额
   - `paypal_price_id = '500'` → $6.99
   - `paypal_price_id = '2000'` → $20.99
   - 其他类型使用 `amount` 字段
3. **Apple IAP**: 根据 `product_id` 映射金额（原价，未计折扣）
   - `product_id` 字段使用下划线分隔（如 `energy_500`）
   - 金额按**原价计算，未计算折扣码优惠**（实际收入可能更低）

In [15]:
interface DailyData {
  date: string;
  stripe_revenue: number;
  paypal_revenue: number;
  iap_revenue: number;
  total_revenue: number;
  total_cost: number;
  gross_profit: number;
  gross_margin: number;
}

// 辅助函数：标准化日期为 YYYY-MM-DD 格式
const normalizeDate = (d: any): string => {
  if (!d) return "";
  const str = String(d);
  // 处理 ISO 格式 (2026-01-10T00:00:00.000Z) 或普通格式 (2026-01-10)
  return str.split("T")[0].split(" ")[0];
};

console.log("📊 查询 Stripe 收入...");
const stripeResult = await mcpClient.callTool({
  name: "bytebase-execute_sql",
  arguments: {
    sql: `-- Stripe 收入（按日汇总）
          SELECT
            DATE(created_date) as revenue_date,
            SUM(amount) as stripe_revenue
          FROM my_shell_prod.user_subscription_stripe_orders
          WHERE status = 'ORDER_STATUS_SUCCESS'
            AND amount >= 0
            AND created_date >= '${startDate} 00:00:00'
            AND created_date <= '${endDate} 23:59:59'
          GROUP BY DATE(created_date)
          ORDER BY revenue_date`
  }
});
const stripeData = JSON.parse((stripeResult.content as any[])[0].text);
const stripeMap = new Map(stripeData.data.rows.map((r: any) => [normalizeDate(r.revenue_date), Number(r.stripe_revenue) || 0]));
console.log(`   ✅ ${stripeData.data.rows.length} 天数据`);

console.log("📊 查询 PayPal 收入...");
const paypalResult = await mcpClient.callTool({
  name: "bytebase-execute_sql",
  arguments: {
    sql: `-- PayPal 收入（按日汇总）
          -- ENERGY 类型订单需特殊处理：paypal_price_id='500'→$6.99，'2000'→$20.99
          SELECT
            DATE(created_date) as revenue_date,
            SUM(
              CASE
                WHEN biz_type = 'ENERGY' AND paypal_price_id = '500' THEN 6.99
                WHEN biz_type = 'ENERGY' AND paypal_price_id = '2000' THEN 20.99
                ELSE amount
              END
            ) as paypal_revenue
          FROM my_shell_prod.user_subscription_paypal_orders
          WHERE status = 'ORDER_STATUS_SUCCESS'
            AND created_date >= '${startDate} 00:00:00'
            AND created_date <= '${endDate} 23:59:59'
          GROUP BY DATE(created_date)
          ORDER BY revenue_date`
  }
});
const paypalData = JSON.parse((paypalResult.content as any[])[0].text);
const paypalMap = new Map(paypalData.data.rows.map((r: any) => [normalizeDate(r.revenue_date), Number(r.paypal_revenue) || 0]));
console.log(`   ✅ ${paypalData.data.rows.length} 天数据`);

console.log("📊 查询 Apple IAP 收入...");
const iapResult = await mcpClient.callTool({
  name: "bytebase-execute_sql",
  arguments: {
    sql: `-- Apple IAP 收入（按日汇总，原价计算）
          SELECT
            DATE(created_date) as revenue_date,
            SUM(
              CASE product_id
                -- 订阅类产品
                WHEN 'PLAYER_MONTHLY' THEN 6.99
                WHEN 'PLAYER_YEARLY' THEN 58.99
                WHEN 'DEVELOPER_MONTHLY' THEN 59.99
                WHEN 'DEVELOPER_YEARLY' THEN 499.99
                WHEN '33OFF_DEVELOPER_MONTHLY' THEN 39.99
                -- 能量类产品
                WHEN 'energy_500' THEN 6.99
                WHEN 'energy_2000' THEN 20.99
                ELSE 0
              END
            ) as iap_revenue
          FROM my_shell_prod.apple_used_transactions
          WHERE created_date >= '${startDate} 00:00:00'
            AND created_date <= '${endDate} 23:59:59'
          GROUP BY DATE(created_date)
          ORDER BY revenue_date`
  }
});
const iapData = JSON.parse((iapResult.content as any[])[0].text);
const iapMap = new Map(iapData.data.rows.map((r: any) => [normalizeDate(r.revenue_date), Number(r.iap_revenue) || 0]));
console.log(`   ✅ ${iapData.data.rows.length} 天数据`);

console.log("✅ 收入查询完成");

📊 查询 Stripe 收入...
   ✅ 5 天数据
📊 查询 PayPal 收入...
   ✅ 0 天数据
📊 查询 Apple IAP 收入...
   ✅ 5 天数据
✅ 收入查询完成


## Step 4: 查询成本

从 `art_task` 表查询每日总成本：
- `actual_energy_cost` 单位是美分，需除以 100 转换为美元
- 包括 `done` 和 `cancel` 状态的任务（已取消的任务也产生了成本）

In [16]:
console.log("📊 查询成本...");
const costResult = await mcpClient.callTool({
  name: "bytebase-execute_sql",
  arguments: {
    sql: `-- 每日总成本（从 art_task 表）
          -- actual_energy_cost 单位是美分，÷100 转为美元
          SELECT
            DATE(created_date) as cost_date,
            SUM(actual_energy_cost) / 100 as total_cost
          FROM my_shell_prod.art_task
          WHERE status IN ('done', 'cancel')
            AND created_date >= '${startDate} 00:00:00'
            AND created_date <= '${endDate} 23:59:59'
          GROUP BY DATE(created_date)
          ORDER BY cost_date`
  }
});
const costData = JSON.parse((costResult.content as any[])[0].text);
const costMap = new Map(costData.data.rows.map((r: any) => [normalizeDate(r.cost_date), Number(r.total_cost) || 0]));
console.log(`   ✅ ${costData.data.rows.length} 天数据`);

console.log("✅ 成本查询完成");

📊 查询成本...
   ✅ 5 天数据
✅ 成本查询完成


## Step 5: 合并数据并计算毛利率

将收入和成本按日期合并，计算：
- **总收入** = Stripe + PayPal + Apple IAP
- **毛利润** = 总收入 - 总成本
- **毛利率** = 毛利润 / 总收入 × 100%

In [17]:
const dailyData: DailyData[] = [];

for (const date of dateList) {
  const stripe = stripeMap.get(date) || 0;
  const paypal = paypalMap.get(date) || 0;
  const iap = iapMap.get(date) || 0;
  const cost = costMap.get(date) || 0;

  const total_revenue = stripe + paypal + iap;
  const gross_profit = total_revenue - cost;
  const gross_margin = total_revenue > 0 ? (gross_profit / total_revenue * 100) : 0;

  dailyData.push({
    date,
    stripe_revenue: stripe,
    paypal_revenue: paypal,
    iap_revenue: iap,
    total_revenue,
    total_cost: cost,
    gross_profit,
    gross_margin
  });
}

// 计算汇总
const summary = {
  total_stripe: dailyData.reduce((sum, d) => sum + d.stripe_revenue, 0),
  total_paypal: dailyData.reduce((sum, d) => sum + d.paypal_revenue, 0),
  total_iap: dailyData.reduce((sum, d) => sum + d.iap_revenue, 0),
  total_revenue: dailyData.reduce((sum, d) => sum + d.total_revenue, 0),
  total_cost: dailyData.reduce((sum, d) => sum + d.total_cost, 0),
  total_profit: 0,
  avg_margin: 0
};
summary.total_profit = summary.total_revenue - summary.total_cost;
summary.avg_margin = summary.total_revenue > 0 ? (summary.total_profit / summary.total_revenue * 100) : 0;

console.log(`\n${"=".repeat(50)}`);
console.log(`📊 毛利率分析报告 (${startDate} 至 ${endDate})`);
console.log(`${"=".repeat(50)}`);
console.log(`总收入:   $${summary.total_revenue.toFixed(2)}`);
console.log(`  - Stripe:  $${summary.total_stripe.toFixed(2)}`);
console.log(`  - PayPal:  $${summary.total_paypal.toFixed(2)}`);
console.log(`  - IAP:     $${summary.total_iap.toFixed(2)} (原价，未计折扣)`);
console.log(`总成本:   $${summary.total_cost.toFixed(2)}`);
console.log(`毛利润:   $${summary.total_profit.toFixed(2)}`);
console.log(`毛利率:   ${summary.avg_margin.toFixed(1)}%`);
console.log(`${"=".repeat(50)}`);


📊 毛利率分析报告 (2026-01-10 至 2026-01-14)
总收入:   $8811.17
  - Stripe:  $7447.77
  - PayPal:  $0.00
  - IAP:     $1363.40 (原价，未计折扣)
总成本:   $4780.12
毛利润:   $4031.05
毛利率:   45.7%


## Step 6: 可视化

生成图表展示：
1. 收入、成本与毛利率趋势（折线图）
2. 收入构成（堆叠柱状图）

In [18]:
import * as Plot from "npm:@observablehq/plot";
import { document } from "jsr:@ry/jupyter-helper";

// 准备图表数据
const trendData = dailyData.flatMap(d => [
  { date: d.date, metric: "总收入", value: d.total_revenue },
  { date: d.date, metric: "总成本", value: d.total_cost },
  { date: d.date, metric: "毛利润", value: d.gross_profit }
]);

const marginData = dailyData.map(d => ({
  date: d.date,
  margin: d.gross_margin
}));

if (Deno.args.length === 0) {
  // 图表1: 收入成本趋势
  Plot.plot({
    document,
    title: "每日收入、成本与毛利润趋势",
    width: 900,
    height: 400,
    x: { label: "日期", tickRotate: -45 },
    y: { label: "金额 (USD)", grid: true },
    color: { legend: true, domain: ["总收入", "总成本", "毛利润"], range: ["#4e79a7", "#e15759", "#59a14f"] },
    marks: [
      Plot.line(trendData, { x: "date", y: "value", stroke: "metric", strokeWidth: 2, tip: true }),
      Plot.dot(trendData, { x: "date", y: "value", fill: "metric", r: 3 }),
      Plot.ruleY([0])
    ]
  });
}

每日收入、成本与毛利润趋势 总收入 总成本 毛利润 0 200 400 600 800 1,000 1,200 1,400 1,600 1,800 ↑ 金额 (USD) 2026-01-10 2026-01-11 2026-01-12 2026-01-13 2026-01-14 日期 ⚠️ 1 warning. Please check the console.

In [19]:
if (Deno.args.length === 0) {
  // 图表2: 毛利率趋势
  Plot.plot({
    document,
    title: "每日毛利率趋势",
    width: 900,
    height: 300,
    x: { label: "日期", tickRotate: -45 },
    y: { label: "毛利率 (%)", grid: true },
    marks: [
      Plot.line(marginData, { x: "date", y: "margin", stroke: "#59a14f", strokeWidth: 2 }),
      Plot.dot(marginData, { x: "date", y: "margin", fill: "#59a14f", r: 4, tip: true }),
      Plot.ruleY([0], { stroke: "#ccc" }),
      Plot.ruleY([summary.avg_margin], { stroke: "#e15759", strokeDasharray: "5,5" })
    ]
  });
}

每日毛利率趋势 0 10 20 30 40 50 ↑ 毛利率 (%) 2026-01-10 2026-01-11 2026-01-12 2026-01-13 2026-01-14 日期 ⚠️ 1 warning. Please check the console.

In [20]:
// 图表3: 收入构成堆叠柱状图
const revenueBreakdown = dailyData.flatMap(d => [
  { date: d.date, source: "Stripe", revenue: d.stripe_revenue },
  { date: d.date, source: "PayPal", revenue: d.paypal_revenue },
  { date: d.date, source: "Apple IAP", revenue: d.iap_revenue }
]);

if (Deno.args.length === 0) {
  Plot.plot({
    document,
    title: "收入构成趋势（注：IAP 为原价，未计折扣）",
    width: 900,
    height: 400,
    x: { label: "日期", tickRotate: -45 },
    y: { label: "收入 (USD)", grid: true },
    color: { legend: true, domain: ["Stripe", "PayPal", "Apple IAP"], range: ["#4e79a7", "#f28e2c", "#76b7b2"] },
    marks: [
      Plot.barY(revenueBreakdown, Plot.stackY({ x: "date", y: "revenue", fill: "source", tip: true })),
      Plot.ruleY([0])
    ]
  });
}

收入构成趋势（注：IAP 为原价，未计折扣） Stripe PayPal Apple IAP 0 200 400 600 800 1,000 1,200 1,400 1,600 1,800 ↑ 收入 (USD) 2026-01-10 2026-01-11 2026-01-12 2026-01-13 2026-01-14 日期 ⚠️ 1 warning. Please check the console.

## Step 7: 导出报告

将每日数据导出为 CSV 文件，并在 Jupyter 模式下显示表格。

In [21]:
import pl from "npm:nodejs-polars";
import { agGrid } from "jsr:@manzt/jupyter-helper";

const df = pl.DataFrame({
  "日期": dailyData.map(d => d.date),
  "Stripe收入": dailyData.map(d => d.stripe_revenue),
  "PayPal收入": dailyData.map(d => d.paypal_revenue),
  "IAP收入": dailyData.map(d => d.iap_revenue),
  "总收入": dailyData.map(d => d.total_revenue),
  "总成本": dailyData.map(d => d.total_cost),
  "毛利润": dailyData.map(d => d.gross_profit),
  "毛利率(%)": dailyData.map(d => Math.round(d.gross_margin * 10) / 10)
});

try { await Deno.mkdir(OUTPUT_DIR, { recursive: true }); } catch {}
const outputFile = `${OUTPUT_DIR}/gross-margin-${startDate}-to-${endDate}.csv`;
df.writeCSV(outputFile);

console.log(`💾 已保存: ${outputFile}`);

if (Deno.args.length === 0) {
  agGrid(df);
}

💾 已保存: ./reports/gross-margin-2026-01-10-to-2026-01-14.csv


---

## 附录

### 关键注意事项

1. **时间范围一致**: 确保收入和成本的时间范围完全一致

2. **单位转换**: `actual_energy_cost` 是美分，需要 ÷ 100

3. **状态过滤**:
   - 收入: 只统计成功订单 (`status = 'ORDER_STATUS_SUCCESS'`)
   - 成本: 包括 `done` 和 `cancel` 状态（已取消任务也产生成本）

4. **数据质量**:
   - PayPal ENERGY 订单需特殊处理（根据 `paypal_price_id` 确定金额）
   - Apple IAP 按原价计算，未计折扣码优惠
   - Apple IAP 表无 `status` 字段，默认所有交易成功
   - 排除退款/测试订单

5. **负毛利率**: 如果出现负毛利率，需要分析：
   - 是否有大量免费用户使用
   - 收入归因是否合理
   - 成本是否异常高

### 相关文档

- `bot-margin-analysis.md` - Bot 级别毛利率分析，使用归因模型分配收入到各个 bot
- `main-site-revenue-attribution.ipynb` - 主站收入归因分析

In [22]:
// CLI 模式下退出进程，避免 MCP 连接保持进程不退出
if (Deno.args.length > 0) Deno.exit(0);